In [1]:
import pickle
import re


In [2]:
def read_sentences(path):
    Sentences = []
    Sentence_Classes = []
    Temp_Sentence = []
    Temp_Sentence_Classes = []
    with open(path, 'r', encoding="utf-8") as file:
        # Iterate through each line
        for line in file:

            # Split the line by the tab character
            parts = line.split('\t')


            # Check if there are at least two parts
            if len(parts) >= 2:
                word = parts[0]
                class_ = parts[1].strip()  # strip() to remove leading/trailing whitespace

                # Do something with the word and class, for example, print them
                #print("Word:", word)
                #print("Class:", class_)


                Temp_Sentence.append(word)
                Temp_Sentence_Classes.append(class_)
                #print(Temp_Sentence)
            else:
                #print("Line does not contain a tab character")
                Sentences.append(Temp_Sentence)
                Sentence_Classes.append(Temp_Sentence_Classes)
                Temp_Sentence = []
                Temp_Sentence_Classes = []
    return Sentences, Sentence_Classes


In [3]:
path_normal_reading = r'C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\zuco1_normalreading_ner.txt'

path_task_reading = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\zuco1_taskreading_ner.txt"

path_sentiment = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\zuco1_sentiment_ner.txt"

normal_reading_sentences, normal_reading_classes = read_sentences(path_normal_reading)
task_reading_sentences, task_reading_classes = read_sentences(path_task_reading)
sentiment_sentences, sentiment_classes = read_sentences(path_sentiment)

#combine
Sentences = [item for sublist in [normal_reading_sentences, task_reading_sentences, sentiment_sentences] for item in sublist]
Sentence_Classes = [item for sublist in [normal_reading_classes, task_reading_classes, sentiment_classes] for item in sublist]


In [4]:
print(Sentences[0])

['Henry', 'Ford', 'advocated', 'long-time', 'associate', 'Harry', 'Bennett', 'to', 'take', 'the', 'spot', '.']


In [5]:
print(Sentence_Classes[0])

['B-PER', 'I-PER', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O']


In [4]:
# To load the lists from the file:
with open(r"C:\Users\gxb18167\PycharmProjects\EEG-To-Text\SIGIR_Development\EEG-GAN\EEG_Text_Pairs_Sentence.pkl", 'rb') as file:
    EEG_word_level_embeddings = pickle.load(file)
    EEG_word_level_labels = pickle.load(file)

In [ ]:
# To load the lists from the file:
with open(r"C:\Users\gxb18167\PycharmProjects\EEG-To-Text\SIGIR_Development\EEG-GAN\Test_EEG_Text_Pairs_Sentence.pkl", 'rb') as file:
    Test_EEG_word_level_embeddings = pickle.load(file)
    Test_EEG_word_level_labels = pickle.load(file)

In [7]:
sentences = []
current_sentence = []

In [50]:
# Iterate through the words
for i in range(len(EEG_word_level_labels)):
    # Check if the word marks the start of a new sentence
    word = EEG_word_level_labels[i]


    if word == "SOS":
        # If it does, append the current sentence to the list of sentences
        if current_sentence:
            sentences.append(current_sentence)
        # Start a new sentence
        current_sentence = []
    else:
        # Add the word to the current sentence
        current_sentence.append(word)


# Append the last sentence
if current_sentence:
    sentences.append(current_sentence)

In [53]:
len(sentences)

10522

In [54]:
# Combine each internal list into a string
combined_sentences = [' '.join(sentence) for sentence in sentences]

10523

In [11]:
# Initialize an empty list to store the list of words for each sentence
list_of_words = []
# Iterate over each sentence in the list
for sentence in combined_sentences:
    # Split the sentence into a list of words including punctuation
    words = re.findall(r'\b\w+\b|[^\w\s]', sentence)
    # Append the list of words to the list of lists
    list_of_words.append(words)


In [55]:
Sentences

In [56]:
Sentence_Classes

In [59]:
List_of_NE = []
List_of_NE_Labels = []
Named_Entity = []
Named_Entity_Label = []


for i in range(len(Sentences)):
    current_sentence = Sentences[i]
    current_sentence_label = Sentence_Classes[i]

    for j in range(len(current_sentence)):
        current_word = current_sentence[j]
        #print(current_word)
        current_word_label = current_sentence_label[j]
        #print(current_word_label)
        if current_word_label != 'O':
            Named_Entity.append(current_word)
            Named_Entity_Label.append(current_word_label)
        else:
            if Named_Entity:
                List_of_NE.append(Named_Entity)
                List_of_NE_Labels.append(Named_Entity_Label)

                Named_Entity = []
                Named_Entity_Label = []

[['Presents',
  'a',
  'case',
  'while',
  'to',
  'provide',
  'a',
  'reason',
  'to',
  'care',
  'beyond',
  'very',
  'basic',
  'dictums',
  'of',
  'human',
  'decency',
  '.'],
 ['Beautifully',
  'crafted',
  ',',
  'engaging',
  'filmmaking',
  'that',
  'should',
  'attract',
  'upscale',
  'audiences',
  'hungry',
  'quality',
  'a',
  'nostalgic',
  ',',
  'twisty',
  'yarn',
  'will',
  'keep',
  'them',
  'guessing',
  '.'],
 ['Bread',
  ',',
  'My',
  'Sweet',
  'has',
  'so',
  'many',
  'flaws',
  'would',
  'easy',
  'critics',
  'to',
  'shred'],
 ['Slow', ',', 'silly', 'and', 'unintentionally', 'hilarious', '.'],
 ['Ultimately',
  'feels',
  'emp11111ty',
  'and',
  'unsatisfying',
  ',',
  'like',
  'Communion',
  'wafer',
  'without',
  'wine',
  '.'],
 ['Exudes',
  'the',
  'Busby',
  'Berkeley',
  'musical',
  'and',
  'visceral',
  'excitement',
  'of',
  'sports',
  'extravaganza',
  '.'],
 ['The',
  'film',
  'rehashes',
  'several',
  'old',
  'themes',
  '

In [58]:
List_of_NE

[['B-PER', 'I-PER', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O'],
 ['B-PER',
  'O',
  'O',
  'B-PER',
  'O',
  'O',
  'O',
  'O',
  'B-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-PER',
  'I-PER',
  'I-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-PER',
  'I-PER',
  'I-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'I-LOC',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-PER',
  'I-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-PER',
  'O',
  'O',
  'O',
  'O',
  'B-PER',
  'I-PER',
  'I-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-PER', 'O', 'O', 'O', 'B-LOC', '

In [16]:
if "Presents" in list_of_words:
    print("Yes")

In [17]:

NE = List_of_NE[0]

if NE in list_of_words:
    print("Yes")


In [18]:
NE

['Henry', 'Ford']

In [19]:
List_of_NE_Labels

[['B-PER', 'I-PER'],
 ['B-PER', 'I-PER'],
 ['B-PER'],
 ['B-PER'],
 ['B-PER'],
 ['B-PER', 'I-PER', 'I-PER'],
 ['B-PER', 'I-PER', 'I-PER'],
 ['B-LOC', 'I-LOC'],
 ['B-PER', 'I-PER'],
 ['B-PER'],
 ['B-PER', 'I-PER', 'I-PER'],
 ['B-PER'],
 ['B-LOC'],
 ['B-LOC'],
 ['B-LOC'],
 ['B-PER', 'I-PER'],
 ['B-PER', 'I-PER'],
 ['B-PER', 'I-PER', 'I-PER'],
 ['B-PER'],
 ['B-PER', 'I-PER'],
 ['B-PER', 'I-PER'],
 ['B-PER'],
 ['B-PER', 'I-PER'],
 ['B-PER', 'I-PER'],
 ['B-ORG'],
 ['B-PER', 'I-PER', 'I-PER'],
 ['B-PER', 'I-PER'],
 ['B-PER'],
 ['B-PER', 'I-PER'],
 ['B-PER'],
 ['B-PER'],
 ['B-PER', 'I-PER', 'I-PER'],
 ['B-ORG', 'I-ORG', 'I-ORG', 'I-ORG'],
 ['B-ORG'],
 ['B-PER'],
 ['B-LOC'],
 ['B-LOC'],
 ['B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG'],
 ['B-ORG', 'I-ORG'],
 ['B-ORG', 'I-ORG'],
 ['B-ORG'],
 ['B-ORG'],
 ['B-LOC', 'I-LOC'],
 ['B-LOC', 'I-LOC'],
 ['B-ORG'],
 ['B-LOC'],
 ['B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG'],
 ['B-LOC', 'B-ORG', 'I-ORG'],
 ['B-PER'],
 ['B-ORG', 'I-ORG', 'I-OR

In [20]:
def is_named_entity_in_sentences(named_entity, sentences):
    counter = 0
    for sentence in sentences:
        for i in range(len(sentence) - len(named_entity) + 1):
            if sentence[i:i+len(named_entity)] == named_entity:
                counter += 1

    if counter > 0:
        return True, counter
    else:
        return False, counter

# Check if each named entity is in the list of sentences
counter_sum = 0
for NE in List_of_NE:
    boolean, counter = is_named_entity_in_sentences(NE, list_of_words)
    if boolean:
        print(f"Named entity {NE} found in {counter} sentences")
    else:
        print(f"Named entity {NE} found in {counter} sentences")
    counter_sum += counter

Named entity ['Henry', 'Ford'] found in 96 sentences
Named entity ['Harry', 'Bennett'] found in 22 sentences
Named entity ['Edsel'] found in 67 sentences
Named entity ['Eleanor'] found in 26 sentences
Named entity ['Edsel'] found in 67 sentences
Named entity ['Henry', 'Ford', 'II'] found in 10 sentences
Named entity ['Aldous', 'Leonard', 'Huxley'] found in 10 sentences
Named entity ['United', 'States'] found in 123 sentences
Named entity ['Huxley', 'family'] found in 8 sentences
Named entity ['Huxley'] found in 163 sentences
Named entity ['F.', 'Matthias', 'Alexander'] found in 0 sentences
Named entity ['Huxley'] found in 163 sentences
Named entity ['Godalming'] found in 12 sentences
Named entity ['Surrey'] found in 12 sentences
Named entity ['England'] found in 170 sentences
Named entity ['Leonard', 'Huxley'] found in 32 sentences
Named entity ['Julia', 'Arnold'] found in 9 sentences
Named entity ['Thomas', 'Henry', 'Huxley'] found in 8 sentences
Named entity ['Darwin'] found in 48 se

In [21]:
counter_sum

157906